## Loading data and functions

In [64]:
import os
import pandas as pd
import numpy as np
from heapq import heappush, heappop, heapify
from or_checker import checker, get_machine_number

from structs import * # classes 
from utils import *   # functions 

## Loading esay testcases

In [15]:
easy_dir = '/Users/yangqingwen/Desktop/Github/OR_Case2/testcase/testcases'
med_dir = '/Users/yangqingwen/Desktop/Github/OR_Case2/testcase/testcases-medium'

easies_list = [f for f in os.listdir(easy_dir)]
meds_list = [f for f in os.listdir(med_dir)]
print(len(easies_list))
print(len(meds_list))

188
153


In [16]:
# 喔乾job id要從1開始XDD 不然make_result會有錯

In [32]:
easies = {}
for i in range(len(easies_list)):
    filename = easies_list[i]
    if filename == '.DS_Store':
        continue
    fullpath = f'{easy_dir}/{filename}'
    df = pd.read_csv(fullpath)
    ### revision ###
    df['Job ID'] = df['Job ID'].apply(lambda x : x+1)
    ###############
    easies[filename] = df

In [45]:
mediums = {}
for i in range(len(meds_list)):
    filename = meds_list[i]
    if filename == '.DS_Store':
        continue
    fullpath = f'{med_dir}/{filename}'
    df = pd.read_csv(fullpath)
    mediums[filename] = df

### Reloading functions

In [25]:
%load_ext autoreload
%autoreload 2

## Heuristic Algorithm with find-hole

In [36]:
def find_hole(job_name, 
              currjob, 
              J,
              M, 
              Mach_Q, 
              AvailMachTable):
    curr_jindex, curr_op = job_name 
    proc_time = currjob.stage_pt[curr_op]
    def find_hole_helper():
        for idx, curritem in enumerate(Mach_Q):
            _, m_id = curritem
            if not AvailMachTable[curr_jindex][curr_op][m_id]:
                continue
            # find legal holes
            if not M.holes[m_id]:
                continue
            for hole_id, hole in enumerate(M.holes[m_id]):
                hole_start, hole_end = hole
                hole_length = hole_end - hole_start 
                # enough length 
                if hole_length - proc_time >= -GAP:
                    # check legal precedence 
                    curr_hole_id = hole_id
                    if curr_op == 1 and hole_start >= currjob.end_time[0]:
                        return idx, m_id, curr_hole_id, hole_start + proc_time
                    elif curr_op == 0: 
                        return idx, m_id, curr_hole_id, hole_start + proc_time
                # 一律從hole_start開始schedule，沒辦法的話就跳過（不然更新holes那邊變超麻煩）
    res = find_hole_helper()
    if not res:
        # print(f'No result in finding a hole for {[x+1 for x in job_name]}')
        return False
    if res:
        idx, m_id, hole_id, fill_end = res
        #print(f'Schduling {m_id+1}, {M.holes[m_id][hole_id]} for {[x+1 for x in job_name]}')
        #print(f'Original: {M.holes[m_id]}')
        # idx是Queue中machine的位置
        hole = M.holes[m_id][hole_id]
        # print(hole)
        hole_start, hole_end = hole
        J.assign(job_name = job_name, 
                mach = m_id, 
                st = hole_start) 
        # update hole length and replace avg_hole_length 
        new_avg_hl = M.schedule_hole(
                job_name = job_name, 
                mach = m_id, 
                hole_id = hole_id, 
                fill_end = fill_end)  
        # print(f'Updated: {M.holes[m_id]}')
        return True 

In [37]:
# while not all operations in all jobs are scheduled
# https://stackoverflow.com/questions/59903948/how-to-iterate-heapq-without-losing-data

def heuristic(J, M): 
    # best_makepsan = sum(job_processing_time) for all jobs / |M|
    # heperparameters 
    TOLRATIO = 0.3
    Fail_Tolerance = 2
    best_makespan = sum(job.stage_pt[0]+job.stage_pt[1] for job in J.jobs)/M.number
    tolerance = best_makespan * TOLRATIO  # tolerance for idle time, if idle > tolerance, do not schedule the curr op in the current epoch. 
    print(f'[INFO] {len(J.jobs)} jobs, {M.number} machines')
    print(f'[INFO] Tolerance: {tolerance:.2f}')

    # Job_Q (lst_ratio, job_index, job_op) 
    Job_Q = make_Q(J)
    # print(f'Job Queue: {Job_Q}')
    # Mach_Q (versatility, avg_hole_length, m)
    Mach_Q = make_mQ(M)
    AvailMachs = getAvailMachs(J = J, M = M)
    
    
    fails = [0 for _ in range(len(J.jobs))]
    epoch = 0
    while Job_Q:
        epoch += 1
        PERMIT = True
        # step 3. extract_min() to get the job with minimal LST and its other attributes
        _, curr_job_index, curr_op = heappop(Job_Q)
        curr_job = J.jobs[curr_job_index]
        op_proc_time = curr_job.stage_pt[curr_op]
        job_name = (curr_job_index, curr_op)
         
        
        # if curr_job has no second operation 
        if op_proc_time <= GAP and curr_op == 1: 
            J.assign(job_name = job_name, 
                    mach = None,
                    st = curr_job.end_time[curr_op-1]) 
            # note that it's only possible for second operation to have proc time = 0
            # so this doesn't trigger index error
            continue 
        # step 4-1. calculate the best machine: find-hole
        # 'job_name', 'currjob', and 'Mach_Q'
        if find_hole(J = J, M = M,
                     job_name = job_name, currjob = curr_job, Mach_Q = Mach_Q,
                  AvailMachTable = AvailMachs):

            continue
            
    
        # step 4-2. if find-hole fails, calculate the best machine and schedule at the end 
        avail_machines_idx = [x-1 for x in curr_job.stage_mach[curr_op]]
        curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x], M.versatile[x], x))
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        if J.completion_times[curr_job_index] + op_proc_time > J.due_dates[curr_job_index] and curr_op == 1 and fails[curr_job_index] < Fail_Tolerance:
            #print(f'[INFO] Job {curr_job_index+1} op {curr_op+1} will be tardy even if scheduled, queue last.')
            curr_new_value = float('inf')
            PERMIT = False
        
        # ARE THERE REASONS TO POSTPONE THE CURR OP (if curr_op is second op)?
        elif M.fintime[curr_machine] < J.completion_times[curr_job_index]:
            
            idle = J.completion_times[curr_job_index] - M.fintime[curr_machine]
            if idle > tolerance and curr_op == 1 and fails[curr_job_index] < Fail_Tolerance:
                #print(f'[INFO] Job {curr_job_index+1} op {curr_op+1} has idle {idle:.2f}, postpone it.')
                PERMIT = False
                if Job_Q:
                    curr_new_value = Job_Q[0][0] + 3
                else:
                    curr_new_value = 0 # the last one 
            else:
                #print(f'[INFO] Job {curr_job_index+1} op {curr_op+1} has idle {idle:.2f}.')
                
                M.add_idle( 
                hole_start = M.fintime[curr_machine],
                hole_end =  J.completion_times[curr_job_index], 
                mach = curr_machine, 
                idle_time = idle)
         
        if PERMIT:
            # print(f'Scheduling {[x+1 for x in job_name]} on machine {curr_machine+1}\'s end at {M.fintime[curr_machine]}')
            J.assign(job_name = job_name, 
                mach = curr_machine, 
                 st = M.fintime[curr_machine]
                ) 
            M._schedule(job_name = job_name, 
               mach = curr_machine, 
                proc_time = op_proc_time,
               st = M.fintime[curr_machine])
            curr_new_value = J.get_LST()[curr_job_index]
        else: 
            fails[curr_job_index] += 1
        # print(f'{epoch} Fails Count:', fails)
        # update the LST value and push it back to Q if the job has its second operation that hasn't been done
        if not PERMIT:
            heappush(Job_Q, (curr_new_value, curr_job_index, curr_op))
            # it maintains the heap invariant, no need to heapify
    return J, M

## Running tests and Checking feasibility

In [69]:
#Results = {}
def run_cases(cases):
    testcase_num = len(cases)
    GAP = 1e-7
    passed, success = 0, 0
    dumps = []
    #######################
    tardies, makespans = 0, 0
    #######################
    for filename, data in cases.items():
        print(f'** Summary {filename}:')
        try:
            M = Machines(data)
            J = Jobs(len(data))
            J.add_jobs(data)
            heuristic(J = J, M = M)
            Tardy_jobs = list(np.where(J.completion_times - J.due_dates > -GAP)[0])
            Tardy_jobs = [x+1 for x in Tardy_jobs]
            success += 1
            Makespan = max(M.fintime)
            ans = make_result(J)
            res = checker(ans, cases[filename])
            #######################
            tardies += len(Tardy_jobs)
            makespans += Makespan
            #######################
            if res:
                passed += 1
                tardy, makespan, sch = res
                print(f'\tTestcase {filename} passed.')
                print(f'\ttardy number: {len(tardy)}, tardies: {tardy}, makespan: {makespan}')
                # print(*sch, sep = '\n')
            else: print(f'\tTestcase {filename} failed.')
        except Exception as e:
            dumps.append(f'{filename} has error {e}.')
    print(f'* {success} testcases executed, {len(cases) - success} dumped') 
    print(*dumps, sep = '\n')
    print(f'* {passed} testcases passed.') 
    return  tardies/len(cases),  makespans/len(cases)

In [70]:
avgt, avgm = run_cases(easies)

** Summary instance 184.csv:
[INFO] 18 jobs, 9 machines
[INFO] Tolerance: 5.15
op1 ends: (18,)
op2 processing time: (18,)
	Testcase instance 184.csv passed.
	tardy number: 5, tardies: [1, 12, 15, 17, 18], makespan: 21.3
** Summary instance 190.csv:
[INFO] 6 jobs, 18 machines
[INFO] Tolerance: 0.74
op1 ends: (6,)
op2 processing time: (6,)
	Testcase instance 190.csv passed.
	tardy number: 1, tardies: [4], makespan: 12.7
** Summary instance 14.csv:
[INFO] 15 jobs, 14 machines
[INFO] Tolerance: 3.31
op1 ends: (15,)
op2 processing time: (15,)
	Testcase instance 14.csv passed.
	tardy number: 3, tardies: [1, 2, 14], makespan: 19.6
** Summary instance 28.csv:
[INFO] 23 jobs, 10 machines
[INFO] Tolerance: 6.55
op1 ends: (23,)
op2 processing time: (23,)
	Testcase instance 28.csv passed.
	tardy number: 12, tardies: [1, 2, 5, 7, 8, 10, 12, 14, 18, 20, 21, 23], makespan: 26.3
** Summary instance 147.csv:
[INFO] 8 jobs, 8 machines
[INFO] Tolerance: 3.18
op1 ends: (8,)
op2 processing time: (8,)
	Test

[INFO] 39 jobs, 12 machines
[INFO] Tolerance: 9.61
op1 ends: (39,)
op2 processing time: (39,)
	Testcase instance 88.csv passed.
	tardy number: 29, tardies: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 28, 29, 31, 32, 33, 34, 35, 38, 39], makespan: 40.5
** Summary instance 118.csv:
[INFO] 24 jobs, 11 machines
[INFO] Tolerance: 6.04
op1 ends: (24,)
op2 processing time: (24,)
	Testcase instance 118.csv passed.
	tardy number: 10, tardies: [7, 10, 11, 13, 14, 15, 18, 21, 23, 24], makespan: 26.3
** Summary instance 77.csv:
[INFO] 4 jobs, 19 machines
[INFO] Tolerance: 0.41
op1 ends: (4,)
op2 processing time: (4,)
	Testcase instance 77.csv passed.
	tardy number: 0, tardies: [], makespan: 13.3
** Summary instance 63.csv:
[INFO] 20 jobs, 7 machines
[INFO] Tolerance: 7.51
op1 ends: (20,)
op2 processing time: (20,)
	Testcase instance 63.csv passed.
	tardy number: 11, tardies: [1, 2, 3, 4, 6, 8, 12, 13, 15, 17, 19], makespan: 30.7
** Summary instance 124.csv:
[INFO] 39 

op1 ends: (74,)
op2 processing time: (74,)
	Testcase instance 112.csv passed.
	tardy number: 58, tardies: [2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 57, 58, 60, 61, 64, 65, 66, 68, 70, 71, 72, 73], makespan: 42.6
** Summary instance 69.csv:
[INFO] 8 jobs, 10 machines
[INFO] Tolerance: 2.29
op1 ends: (8,)
op2 processing time: (8,)
	Testcase instance 69.csv passed.
	tardy number: 0, tardies: [], makespan: 16.3
** Summary instance 106.csv:
[INFO] 16 jobs, 4 machines
[INFO] Tolerance: 9.52
op1 ends: (16,)
op2 processing time: (16,)
	Testcase instance 106.csv passed.
	tardy number: 12, tardies: [1, 2, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15], makespan: 41.7
** Summary instance 68.csv:
[INFO] 22 jobs, 16 machines
[INFO] Tolerance: 3.93
op1 ends: (22,)
op2 processing time: (22,)
	Testcase instance 68.csv passed.
	tardy number: 5, tardies: [7, 12, 18, 19, 21], make

op1 ends: (47,)
op2 processing time: (47,)
	Testcase instance 160.csv passed.
	tardy number: 22, tardies: [2, 3, 4, 14, 15, 16, 17, 20, 22, 25, 26, 28, 29, 30, 31, 34, 35, 36, 37, 39, 44, 47], makespan: 27.8
** Summary instance 148.csv:
[INFO] 33 jobs, 14 machines
[INFO] Tolerance: 7.52
op1 ends: (33,)
op2 processing time: (33,)
	Testcase instance 148.csv passed.
	tardy number: 17, tardies: [2, 4, 7, 8, 11, 13, 15, 16, 18, 19, 24, 26, 27, 29, 31, 32, 33], makespan: 32.1
** Summary instance 27.csv:
[INFO] 4 jobs, 13 machines
[INFO] Tolerance: 0.84
op1 ends: (4,)
op2 processing time: (4,)
	Testcase instance 27.csv passed.
	tardy number: 0, tardies: [], makespan: 17.0
** Summary instance 33.csv:
[INFO] 9 jobs, 17 machines
[INFO] Tolerance: 1.52
op1 ends: (9,)
op2 processing time: (9,)
	Testcase instance 33.csv passed.
	tardy number: 1, tardies: [6], makespan: 15.8
** Summary instance 6.csv:
[INFO] 51 jobs, 15 machines
[INFO] Tolerance: 9.28
op1 ends: (51,)
op2 processing time: (51,)
	Test

In [71]:
print('EASY cases:')
print('average tardy job for all these cases:', avgt)
print('average makespans for all these cases:', avgm)

12.823529411764707
24.82085561497325


In [61]:
avgt, avgm = run_cases(mediums)

** Summary instance 14.csv:
[INFO] 29 jobs, 8 machines
[INFO] Tolerance: 9.53
op1 ends: (29,)
op2 processing time: (29,)
REVISED!!!!
	Testcase instance 14.csv passed.
	tardy number: 21, tardies: [3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 16, 17, 18, 21, 22, 24, 25, 26, 27, 28, 29], makespan: 38.2
** Summary instance 28.csv:
[INFO] 17 jobs, 16 machines
[INFO] Tolerance: 2.54
op1 ends: (17,)
op2 processing time: (17,)
REVISED!!!!
	Testcase instance 28.csv passed.
	tardy number: 2, tardies: [12, 14], makespan: 17.8
** Summary instance 147.csv:
[INFO] 5 jobs, 4 machines
[INFO] Tolerance: 4.04
op1 ends: (5,)
op2 processing time: (5,)
REVISED!!!!
	Testcase instance 147.csv passed.
	tardy number: 1, tardies: [2], makespan: 17.8
** Summary instance 153.csv:
[INFO] 31 jobs, 15 machines
[INFO] Tolerance: 5.23
op1 ends: (31,)
op2 processing time: (31,)
REVISED!!!!
	Testcase instance 153.csv passed.
	tardy number: 11, tardies: [6, 10, 12, 14, 15, 16, 18, 24, 25, 28, 31], makespan: 28.5
** Summary instance

[INFO] 15 jobs, 9 machines
[INFO] Tolerance: 3.77
op1 ends: (15,)
op2 processing time: (15,)
REVISED!!!!
	Testcase instance 76.csv passed.
	tardy number: 3, tardies: [2, 12, 14], makespan: 17.8
** Summary instance 89.csv:
[INFO] 26 jobs, 8 machines
[INFO] Tolerance: 8.31
op1 ends: (26,)
op2 processing time: (26,)
REVISED!!!!
	Testcase instance 89.csv passed.
	tardy number: 15, tardies: [1, 3, 4, 5, 9, 10, 12, 14, 16, 17, 18, 22, 24, 25, 26], makespan: 36.3
** Summary instance 60.csv:
[INFO] 14 jobs, 5 machines
[INFO] Tolerance: 5.90
op1 ends: (14,)
op2 processing time: (14,)
REVISED!!!!
	Testcase instance 60.csv passed.
	tardy number: 8, tardies: [3, 6, 8, 9, 10, 12, 13, 14], makespan: 22.8
** Summary instance 74.csv:
[INFO] 21 jobs, 9 machines
[INFO] Tolerance: 5.64
op1 ends: (21,)
op2 processing time: (21,)
REVISED!!!!
	Testcase instance 74.csv passed.
	tardy number: 10, tardies: [3, 7, 9, 10, 12, 13, 14, 15, 16, 18], makespan: 22.8
** Summary instance 133.csv:
[INFO] 57 jobs, 17 mac

[INFO] 56 jobs, 13 machines
[INFO] Tolerance: 10.98
op1 ends: (56,)
op2 processing time: (56,)
REVISED!!!!
	Testcase instance 95.csv passed.
	tardy number: 44, tardies: [1, 3, 4, 6, 9, 10, 12, 13, 14, 16, 17, 18, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56], makespan: 40.5
** Summary instance 81.csv:
[INFO] 42 jobs, 15 machines
[INFO] Tolerance: 6.93
op1 ends: (42,)
op2 processing time: (42,)
REVISED!!!!
	Testcase instance 81.csv passed.
	tardy number: 25, tardies: [1, 6, 7, 9, 10, 12, 14, 15, 16, 17, 18, 24, 25, 27, 28, 30, 31, 32, 33, 34, 35, 36, 38, 39, 41], makespan: 32.4
** Summary instance 139.csv:
[INFO] 55 jobs, 15 machines
[INFO] Tolerance: 9.25
op1 ends: (55,)
op2 processing time: (55,)
REVISED!!!!
	Testcase instance 139.csv passed.
	tardy number: 39, tardies: [3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 45, 46, 47, 4

	Testcase instance 116.csv passed.
	tardy number: 3, tardies: [12, 14, 18], makespan: 17.8
** Summary instance 92.csv:
[INFO] 35 jobs, 11 machines
[INFO] Tolerance: 8.03
op1 ends: (35,)
op2 processing time: (35,)
REVISED!!!!
	Testcase instance 92.csv passed.
	tardy number: 22, tardies: [3, 4, 6, 7, 9, 12, 13, 14, 16, 17, 18, 22, 24, 25, 27, 28, 29, 31, 32, 33, 34, 35], makespan: 34.5
** Summary instance 86.csv:
[INFO] 8 jobs, 4 machines
[INFO] Tolerance: 4.61
op1 ends: (8,)
op2 processing time: (8,)
REVISED!!!!
	Testcase instance 86.csv passed.
	tardy number: 3, tardies: [3, 4, 7], makespan: 19.5
** Summary instance 90.csv:
[INFO] 16 jobs, 14 machines
[INFO] Tolerance: 2.59
op1 ends: (16,)
op2 processing time: (16,)
REVISED!!!!
	Testcase instance 90.csv passed.
	tardy number: 2, tardies: [12, 14], makespan: 17.8
** Summary instance 84.csv:
[INFO] 4 jobs, 9 machines
[INFO] Tolerance: 1.50
op1 ends: (4,)
op2 processing time: (4,)
REVISED!!!!
	Testcase instance 84.csv passed.
	tardy numbe

In [ ]:
print('MEDIUM cases:')
print('average tardy job for all these cases:', avgt)
print('average makespans for all these cases:', avgm)